Question-3

Decision Tree Node Structure:

In [2]:
import numpy as np
import pandas as pd

Initializing a Node:

Parameters:
        - feature_index: Index of feature used for splitting (None for leaf nodes)
        - threshold: Threshold value for splitting (None for leaf nodes)
        - left: Left child node (None for leaf nodes)
        - right: Right child node (None for leaf nodes)
        - value: Predicted class value (None for internal nodes)

In [3]:
class Node:
    def __init__(self,feature_index=None, threshold=None, left=None,right=None, value=None):
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value
        
    def is_leaf(self):
        return self.value is not None
    def set_as_leaf(self, value):
        (self.feature_index,self.threshold,self.right,self.left,self.value)=(None,None,None,None,value)
        
    def set_as_internal(self, feature_index, threshold, left, right):
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = None

def print_tree(node, depth=0, feature_names=None):
    indent = "    "*depth
    
    if node.is_leaf():
        print(f"{indent}Leaf: Class {node.value}")
    else:
        if feature_names is not None:
            feature_names = list(feature_names)
        if feature_names:
            feature_name = feature_names[node.feature_index]
        else:
            feature_name = f"Feature {node.feature_index}"
            
        print(f"{indent}{feature_name} <= {node.threshold}")
        print_tree(node.left, depth + 1, feature_names)
        print_tree(node.right, depth + 1, feature_names)

Binary Spliting

Recursively builds the decision tree.
    
    Parameters:
    - X: Feature matrix
    - y: Target labels
    - current_depth: Current depth of the tree
    - max_depth: Maximum allowed depth (None for no limit)
    - min_samples_split: Minimum number of samples required to split a node
    
    Returns:
    - Node: The root node of the (sub)tree

In [4]:
def build_tree(X, y, current_depth=0, max_depth=None, min_samples_split=2):
    n_samples = X.shape[0]
    node = Node()
    
    # Checking stopping conditions
    if ((len(np.unique(y)) == 1) or (max_depth is not None and current_depth >= max_depth) or (n_samples < min_samples_split)):
        # Make this a leaf node with the majority class
        unique_classes, counts = np.unique(y, return_counts=True)
        majority_class = unique_classes[np.argmax(counts)]
        node.set_as_leaf(majority_class)
        return node #To be checked at end...
    
    #finding best split
    best_split = find_best_split(X, y)
    
    # Split the data
    X_left = X[best_split['left_indices']]
    y_left = y[best_split['left_indices']]
    X_right = X[best_split['right_indices']]
    y_right = y[best_split['right_indices']]
    
    left_child = build_tree(X_left, y_left, current_depth + 1, max_depth, min_samples_split)
    right_child = build_tree(X_right, y_right, current_depth + 1, max_depth, min_samples_split)
    
    node.set_as_internal(best_split['feature_index'], best_split['threshold'], 
                        left_child, right_child)
    
    return node

Impurity Matrics

~Gini Index, Weighted Gini Index

In [6]:
def gini_impurity(y):
    #Hard-coded for two classes..
    if len(y) == 0:
        return 0
    n0 = np.sum(y == 0) / len(y)
    n1 = np.sum(y == 1) / len(y)
    return 1 - (n0**2 + n1**2)

def gini_index_wt(y_left, y_right):
    n_left = len(y_left)
    n_right = len(y_right)
    n_total = n_left + n_right
    if n_total == 0:
        return 0
    
    gini_left = gini_impurity(y_left)
    gini_right = gini_impurity(y_right)
    left_frac=n_left/n_total*gini_left
    right_frac=n_right/n_total*gini_right
    weighted_gini =(left_frac)+(right_frac)
    return weighted_gini

Best Split Find using gini index:

In [8]:
def find_best_split(X, y):
    best_split = {
        'left_indices': None,
        'right_indices': None,
        'gini': float('inf'),
        'feature_index': None,
        'threshold': None
    }
    
    n_samples, n_features = X.shape
    if n_samples <= 1: #Not Sufficient Samples
        return best_split
    
    for feature_index in range(n_features):
        f_val = np.unique(X[:, feature_index])
        
        for threshold in f_val:
            left_indices = np.where(X[:, feature_index] <= threshold)[0]
            right_indices = np.where(X[:, feature_index] > threshold)[0]
            if len(left_indices) == 0 or len(right_indices) == 0:
                continue
            currgini = gini_index_wt(y[left_indices], y[right_indices])
            
            #Updating this if found better..
            if currgini < best_split['gini']:
                best_split = {
                    'left_indices': left_indices,
                    'right_indices': right_indices,
                    'gini': currgini,
                    'feature_index': feature_index,
                    'threshold': threshold,
                }
    return best_split

Prediction Of Samples:

Prediction Function:
One for entire feature matrix
One for single input array.

In [9]:
def predict(tree, X):
    if len(X.shape) == 1:
        X = X.reshape(1, -1)
    
    predictions = []
    for sample in X:
        predictions.append(_predict_sample(tree, sample))
    
    return np.array(predictions)

def _predict_sample(node, sample):
    if node.is_leaf():
        return node.value
    
    if sample[node.feature_index] <= node.threshold:
        return _predict_sample(node.left, sample)
    else:
        return _predict_sample(node.right, sample)

Training On DataSet

To make all features into integers:
Income(Low:0, Medium:1,High:2)
Student(Yes:1,NO:0)
Credit(Fair:0,Excellent:1)
Buy_Computer(Yes:1,No:0)

Making changes to data too.

In [10]:
data = {
    'Age': [25, 30, 35, 40, 45, 50, 55, 60],
    'Income': ['High', 'High', 'Medium', 'Low', 'Low', 'Low', 'Medium', 'High'],
    'Student': ['No', 'No', 'No', 'No', 'Yes', 'Yes', 'Yes', 'No'],
    'Credit Rating': ['Fair', 'Excellent', 'Fair', 'Fair', 'Fair', 'Excellent', 'Excellent', 'Fair'],
    'Buy Computer': ['No', 'No', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'No']
}

df = pd.DataFrame(data)

income_mapping = {'Low': 0, 'Medium': 1, 'High': 2}
df['Income'] = df['Income'].map(income_mapping)

student_mapping = {'No': 0, 'Yes': 1}
df['Student'] = df['Student'].map(student_mapping)

credit_mapping = {'Fair': 0, 'Excellent': 1}
df['Credit Rating'] = df['Credit Rating'].map(credit_mapping)

target_mapping = {'No': 0, 'Yes': 1}
df['Buy Computer'] = df['Buy Computer'].map(target_mapping)

X = df.drop('Buy Computer', axis=1).values
y = df['Buy Computer'].values

#prepared data
print("Features (X):")
print(X)
print("\nTarget (y):", y)


Features (X):
[[25  2  0  0]
 [30  2  0  1]
 [35  1  0  0]
 [40  0  0  0]
 [45  0  1  0]
 [50  0  1  1]
 [55  1  1  1]
 [60  2  0  0]]

Target (y): [0 0 1 1 1 0 1 0]


Predict whether a new person (Age = 42, Income = Low, Student = No,
 Credit = Excellent) will buy a computer.
 Making Prediction..

In [11]:

decision_tree = build_tree(X, y, max_depth=3)
print("Decision Tree Structure:")

print_tree(decision_tree, feature_names=list(df.columns[:-1])) # Exclude target column
test_sample = np.array([42, 0, 0, 1]) #Numerical representation
prediction = predict(decision_tree, test_sample)
verdict = "Yes" if prediction[0] == 1 else "No"
print(f"Prediction for new sample: {verdict}")


Decision Tree Structure:
Income <= 1
    Age <= 45
        Leaf: Class 1
        Age <= 50
            Leaf: Class 0
            Leaf: Class 1
    Leaf: Class 0
Prediction for new sample: Yes


Bagging: To Improve Performance

1) BootStrap Sample building
Create a bootstrap sample of the dataset (sampling with replacement).
    
    Parameters:
    X (numpy array): Feature matrix
    y (numpy array): Target labels
    
    Returns:
    tuple: (X_sample, y_sample, oob_indices)
        - X_sample: Bootstrap sampled feature matrix
        - y_sample: Corresponding labels
        - oob_indices: Indices of out-of-bag samples

In [12]:
def bootstrap_sample(X, y):
    n_samples = X.shape[0]
    indices = np.random.choice(n_samples, size=n_samples, replace=True)
    oob_indices = np.setdiff1d(np.arange(n_samples), np.unique(indices))
    
    X_sample = X[indices]
    y_sample = y[indices]
    
    return X_sample, y_sample, oob_indices

X_sample, y_sample, oob_indices = bootstrap_sample(X, y)
# Test bootstrap sampling
print("Original dataset size:", X.shape)
print("\nBootstrap sample size:", X_sample.shape)
print("Number of out-of-bag samples:", len(oob_indices))
print("Out-of-bag indices:", oob_indices)

Original dataset size: (8, 4)

Bootstrap sample size: (8, 4)
Number of out-of-bag samples: 4
Out-of-bag indices: [2 4 5 7]


Building Multiple Trees for bagging
Build an ensemble of decision trees using bootstrap sampling.
    
    Parameters:
    X (numpy array): Feature matrix
    y (numpy array): Target labels
    n_trees (int): Number of trees in ensemble
    max_depth (int): Maximum depth for each tree
    min_samples_split (int): Minimum samples to split a node
    Random Forest:1->for later use,0->for now where we have to consider all possible cuts.
    
    Returns:
    tuple: (trees, oob_info)
        - trees: List of decision trees
        - oob_info: Dictionary mapping tree indices to their OOB samples

In [13]:
def build_bagged_trees(X, y, n_trees=10, max_depth=3, min_samples_split=2, random_forest=0):
    trees = []
    oob_info = {}
    
    for i in range(n_trees):
        # Create a bootstrap sample
        X_sample, y_sample, oob_indices = bootstrap_sample(X, y)
        
        # Build tree on bootstrap sample
        if(random_forest==1):
            tree = build_tree_2(X_sample, y_sample, 
                         max_depth=max_depth, 
                         min_samples_split=min_samples_split)
        else:
            tree = build_tree(X_sample, y_sample, 
                            max_depth=max_depth, 
                            min_samples_split=min_samples_split)
        
        trees.append(tree)
        oob_info[i] = oob_indices
    
    return trees, oob_info

OOB Error Calculation

Description:
Compute Out-of-Bag error using ensemble and OOB information.
    Parameters:
    ensemble: Collection of decision trees
    oob_info: Dictionary mapping tree indices to their OOB samples
    X: Full feature matrix
    y: True labels
    Returns:
    float: OOB error rate

In [14]:
def compute_oob_error(ensemble, oob_info, X, y):
    n_samples = X.shape[0]
    oob_predictions = {i: [] for i in range(n_samples)}
    #Dict. for oob predictions.
    for tree_idx, tree in enumerate(ensemble):
        oob_indices = oob_info[tree_idx]
        if len(oob_indices) > 0:
            predictions = predict(tree, X[oob_indices])
            # Store predictions
            for sample_idx, pred in zip(oob_indices, predictions):
                oob_predictions[sample_idx].append(pred)
    
    # computing error rate
    errors = 0
    valid_samples = 0
    
    for sample_idx in range(n_samples):
        if oob_predictions[sample_idx]:  # If sample has OOB predictions
            majority_vote = np.round(np.mean(oob_predictions[sample_idx]))
            errors += (majority_vote != y[sample_idx])
            valid_samples += 1
    if(valid_samples==0):
        return 0
    error=errors/valid_samples
    return error

Q4) a
Improve the performance by bagging 10 diferent trees. Compute the OOB
 error.

In [16]:
#Build ensemble with OOB tracking
trees, oob_info = build_bagged_trees(X, y, n_trees=10)
oob_error = compute_oob_error(trees, oob_info, X, y)
test_sample = np.array([42, 0, 0, 1]).reshape(1, -1)
# Get predictions from all trees
all_predictions = np.array([predict(tree, test_sample)[0] for tree in trees])
verdict = np.round(np.mean(all_predictions))
action='Buy' if verdict == 1 else "Not buy"
print(f"\nAfter Bagging Prediction: {action} computer")
print(f"OOB Error Estimate: {oob_error:.3%}")



After Bagging Prediction: Buy computer
OOB Error Estimate: 75.000%


Bagging With Random Feature Selection(Random Forest)

We will build 10 decision trees, and at every node we will consider two randomly selected features for split(Without Replacement).
Build tree methods needs to be changed, along with best split too..

In [17]:
def build_tree_2(X, y, current_depth=0, max_depth=None, min_samples_split=2, n_random_features=2):
    n_samples, n_features = X.shape
    node = Node()
    
    # Stopping conditions
    if ((len(np.unique(y)) == 1) or (max_depth is not None and current_depth >= max_depth) or (n_samples < min_samples_split)):
        unique_classes, counts = np.unique(y, return_counts=True)
        majority_class = unique_classes[np.argmax(counts)]
        node.set_as_leaf(majority_class)
        return node #Making root node
    
    # Random features to consider for this split
    feature_indices = np.random.choice(n_features, size=min(n_random_features, n_features), replace=False)
    
    best_split = {
        'feature_index': None,
        'threshold': None,
        'left_indices': None,
        'right_indices': None,
        'gini': float('inf')
    }
    
    for feature_index in feature_indices:
        f_val = np.unique(X[:, feature_index])
        
        for threshold in f_val:
            left_indices = np.where(X[:, feature_index] <= threshold)[0]
            right_indices = np.where(X[:, feature_index] > threshold)[0]
            
            if len(left_indices)==0 or len(right_indices)==0:
                continue
                
            currgini =gini_index_wt(y[left_indices], y[right_indices])
            
            if currgini < best_split['gini']:
                best_split = {
                    'feature_index':feature_index,
                    'threshold': threshold,
                    'left_indices': left_indices,
                    'right_indices':right_indices,
                    'gini': currgini
                }

    if best_split['feature_index'] is None:
        unique_classes, counts = np.unique(y, return_counts=True)
        majority_class = unique_classes[np.argmax(counts)]
        node.set_as_leaf(majority_class)
        return node
    
    X_left = X[best_split['left_indices']]
    y_left = y[best_split['left_indices']]
    X_right = X[best_split['right_indices']]
    y_right = y[best_split['right_indices']]
    
    left_child = build_tree_2(X_left, y_left, current_depth + 1, max_depth, min_samples_split, n_random_features)
    right_child = build_tree_2(X_right, y_right, current_depth + 1, max_depth, min_samples_split, n_random_features)
    
    node.set_as_internal(best_split['feature_index'], best_split['threshold'], left_child, right_child)
    return node

Improve the performance by bagging 10 diferent trees but using only two
random predictors while building the trees. Compute the OOB error.

In [21]:
# Building ensemble of 10 trees, each using 2 random features per split
trees, oob_info = build_bagged_trees(X, y, n_trees=10,random_forest=1)
oob_error = compute_oob_error(trees, oob_info, X, y)
all_predictions = np.array([predict(tree, test_sample)[0] for tree in trees])
verdict = np.round(np.mean(all_predictions))
action='Buy' if verdict == 1 else "Not buy"
print(f"\nRandom Forest Prediction: {action} computer")
print(f"OOB Error with random feature selection: {oob_error:.3%}")


Random Forest Prediction: Buy computer
OOB Error with random feature selection: 75.000%
